In [1]:
import sys
from pathlib import Path

# Add parent directory to sys.path
parent_dir = Path().resolve().parent  # current folder's parent
sys.path.append(str(parent_dir))

import geopandas as gpd 
from shapely import wkt
import pandas as pd
import numpy as np
import isochrones 
import population 
import h3_utils
import geoutils
import api_keys

import os
import re
import time
from tqdm import tqdm

In [2]:
aoi = geoutils.get_city_geometry("Ghana")
countrycode = "GH"
batch_size = 50
data_path = "../data/Virtue-Foundation-Ghana-enriched.csv"
geocoded_path = "../data/Virtue-Foundation-Ghana-enriched_geocoded.csv"

In [6]:
# Load existing geocoded data or original data
if os.path.isfile(geocoded_path):
    data = pd.read_csv(geocoded_path)
else:
    data = pd.read_csv(data_path)

    # Function to concatenate address parts
    def concat_address(row, level=0):
        cols = [
            'name',
            'address_line1',
            'address_line2',
            'address_line3',
            'address_city',
            'address_stateOrRegion',
            'address_zipOrPostcode',
            'address_country'
        ]
        if level == 1:
            cols = cols[1:]
        elif level == 2:
            cols = cols[4:]

        parts = [str(row[c]).strip() for c in cols if pd.notna(row[c]) and str(row[c]).strip()]
        return " , ".join(parts)

    # Create address columns
    data['address_and_name'] = data.apply(concat_address, axis=1)
    data['address_complete'] = data.apply(lambda x: concat_address(x, level=1), axis=1)
    data['address_only_city'] = data.apply(lambda x: concat_address(x, level=2), axis=1)

    # Initialize new columns
    data["geometry"] = None
    data["geometry_source"] = None

# Geocode function
def geocode(row):
    for col in ["address_and_name", "address_complete", "address_only_city"]:
        try:
            geo = geoutils.get_address_point_opencage(row[col], api_keys.OPENCAGE, countrycode=countrycode)
            if geo:  # success
                return pd.Series({"geometry": geo, "geometry_source": col})
        except Exception as e:
            if "You have used the requests available on your plan" in str(e):
                raise  # stop execution if API limit reached
            else:
                continue  # try next column

    # if none worked
    return pd.Series({"geometry": None, "geometry_source": None})

# Process in batches
for start in tqdm(range(0, len(data), batch_size), desc="Geocoding batches"):
    end = start + batch_size
    batch = data.iloc[start:end].copy()

    # Only geocode rows where geometry is None
    mask = batch["geometry"].isna()
    if mask.any():
        batch.loc[mask, ["geometry", "geometry_source"]] = batch.loc[mask].apply(geocode, axis=1)

        # Update the main DataFrame
        data.loc[start:end-1, ["geometry", "geometry_source"]] = batch[["geometry", "geometry_source"]]

    data.to_csv(geocoded_path)
    print(f"Processed rows {start} to {end-1}")
    time.sleep(1)  # avoid API throttling

# Save complete CSV at the end
data.to_csv(geocoded_path)

Geocoding batches:   0%|          | 0/16 [00:00<?, ?it/s]

Processed rows 0 to 49


Geocoding batches:   6%|▋         | 1/16 [00:01<00:15,  1.03s/it]

Processed rows 50 to 99


Geocoding batches:  12%|█▎        | 2/16 [00:02<00:14,  1.03s/it]

...

Geocoding batches:  88%|████████▊ | 14/16 [00:14<00:02,  1.03s/it]

Processed rows 700 to 749


Geocoding batches:  94%|█████████▍| 15/16 [00:15<00:01,  1.03s/it]

Processed rows 750 to 799


Geocoding batches: 100%|██████████| 16/16 [00:16<00:00,  1.03s/it]


In [14]:
# Function to extract the WKT from messy string
def extract_wkt(s):
    if isinstance(s, str):
        match = re.search(r'(POINT|LINESTRING|POLYGON)\s*\(.*\)', s)
        if match:
            return match.group(0)
    return None

data["geometry"] = data["geometry"].apply(lambda x: wkt.loads(extract_wkt(x)) if extract_wkt(x) else None)
data = gpd.GeoDataFrame(data,geometry="geometry",crs=4326)
data = data[data.geometry.intersects(aoi.to_crs(4326).union_all())]
data = data[[col for col in data.columns if "Unnamed:" not in col]]
data["geometry"] = data.geometry.to_wkt().astype(str)
data.to_csv(geocoded_path)